In [1]:
sc.install_pypi_package("numpy==1.19.2")
sc.install_pypi_package("pip==20.2.4")
sc.install_pypi_package("pandas==0.23.2")
sc.install_pypi_package("opencv-python")
sc.install_pypi_package("pyarrow")
sc.install_pypi_package("scikit-learn")
sc.install_pypi_package("Pillow")
sc.install_pypi_package("pyquickhelper")
sc.install_pypi_package("sparkdl")
sc.install_pypi_package("findspark")
sc.install_pypi_package("wheel==0.35")
sc.install_pypi_package("flatbuffers==1.12.0")
sc.install_pypi_package("gast==0.3.3")
sc.install_pypi_package("grpcio==1.32.0")
sc.install_pypi_package("opt-einsum==3.3.0")
sc.install_pypi_package("six==1.15.0")
sc.install_pypi_package("typing-extensions ==3.7.4")
sc.install_pypi_package("setuptools==41.0.0")
sc.install_pypi_package("tensorflow-gpu")
sc.install_pypi_package("tensorframes")
sc.list_packages()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1644571077966_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid package name: Package name and version must contain valid characters
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 1293, in install_pypi_package
    pypi_package = self._validate_package(pypi_package)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 1366, in _validate_package
    raise ValueError("Invalid package name: Package name and version must contain valid characters")
ValueError: Invalid package name: Package name and version must contain valid characters



In [2]:
sc.install_pypi_package("keras")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)

You should consider upgrading via the '/tmp/1644571666112-0/bin/python -m pip install --upgrade pip' command.

In [3]:
import numpy as np
import pandas as pd

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# librairie python
import sys
import os
import time
import io

# librairie pyspark
import pyspark
from pyspark.sql import SparkSession, Row, SQLContext
from pyspark import SparkContext
from pyspark.sql.functions import element_at, split
from pyspark.sql.types import *
from pyspark.ml.image import ImageSchema
from pyspark.ml.feature import PCA
from pyspark.sql.functions import col, pandas_udf, PandasUDFType
from pyspark.ml import Pipeline 
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.ml.linalg import Vectors, VectorUDT, DenseVector
from pyspark.sql import types
from pyspark.ml.functions import array_to_vector

# librairie traitement d'image
from PIL import Image 
import cv2



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
os.environ['PYSPARK_PYTHON']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'/tmp/1644571666112-0/bin/python'

In [6]:
import tensorflow

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# librairie deep learning

from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Model

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# D'apres la documentation :
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
spark.conf.set('spark.sql.repl.eagerEval.maxNumRows', 5)
# Enable Arrow-based columnar data transfers
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
S3 = True

if S3 is True :
    spark = SparkSession.builder.appName('Projet_8_Application').getOrCreate()
    sc = SparkSession.sparkContext

else :

    spark = SparkSession.builder\
            .master("local[*]")\
            .appName('Projet_8')\
            .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Chargement des fichiers

In [10]:
def chargement_fichier() :
    
    if S3 is True :
        
        train_path = "s3://compartimentduprojet8/Images/*"

        
        image_training = spark.read.format("binaryFile") \
                                   .option("pathGlobFilter", "*.jpg") \
                                   .option("recursiveFileLookup", "true") \
                                   .load(train_path)
        
        images_training_im = spark.read.format("image").load(train_path, recursive=True)
        
        image_training = image_training.withColumn('classe', element_at(split(image_training['path'], "/"), -2))
        
    else :
        train_path = "C://Train/Apple Braeburn/*"
        test_path = "C://fruits-360/Test/*"
        
        image_training = spark.read.format("binaryFile") \
                               .option("pathGlobFilter", "*.jpg") \
                               .option("recursiveFileLookup", "true") \
                               .load(train_path)
        
        image_training = image_training.withColumn('classe', element_at(split(image_training['path'], "/"), -2))
        
    return image_training, images_training_im

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
image_training, images_training_im = chargement_fichier()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
image_training = image_training.sample(0.5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
image_training.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

33723

# Extraction des features

In [14]:
Mob = MobileNet(weights=None, include_top=False, pooling = 'avg', input_shape=(224, 224,3))

bc_model_weights = spark.sparkContext.broadcast(Mob.get_weights())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
def model_fn():
    """
    Returns a mobilenet model with top layer removed and broadcasted pretrained weights.
    """
    model = MobileNet(weights=None, include_top=False, pooling = 'avg')
    model.set_weights(bc_model_weights.value)
    
    return model

model = model_fn()



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    # lecture + redimension (224x224) pour Xception
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img) # image en array
    return preprocess_input(arr)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)

def featurize_udf(content_series_iter) :
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/pandas/functions.py:392: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

In [19]:
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")


def extraction_features(image_training):
    
    features_df = image_training.repartition(16).select(col("path"), featurize_udf("content").alias("features"))
    
    return features_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
features = extraction_features(image_training)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Réduction dimensionnelle

In [21]:
to_vector_udf = udf(lambda r: Vectors.dense(r), VectorUDT())
df_with_vectors = features.withColumn('features_pca', to_vector_udf('features'))
df_with_vectors = df_with_vectors.select('path','features_pca')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
pca = PCA(k=10, inputCol= "features_pca", outputCol='vectors_pca')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
model_pca = pca.fit(df_with_vectors)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
data = model_pca.transform(df_with_vectors)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
data.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+
|                path|        features_pca|         vectors_pca|
+--------------------+--------------------+--------------------+
|s3://compartiment...|[3.12977416275880...|[-3.8270434711065...|
|s3://compartiment...|[2.37882260573779...|[-3.7267892628358...|
|s3://compartiment...|[4.02374294547856...|[-5.4888031134205...|
|s3://compartiment...|[3.28254488993325...|[-5.3337637980856...|
|s3://compartiment...|[2.00557007707192...|[-2.2085868166513...|
|s3://compartiment...|[4.65291114044995...|[-6.7360928650660...|
|s3://compartiment...|[4.39744077034821...|[-6.1952343241092...|
|s3://compartiment...|[3.80907422844327...|[-5.9497329437110...|
|s3://compartiment...|[4.73620192734422...|[-7.1441352072730...|
|s3://compartiment...|[4.72118544883147...|[-5.7318825719117...|
|s3://compartiment...|[5.32986355608049...|[-8.4505086848692...|
|s3://compartiment...|[3.95073940965935...|[-6.3184442014092...|
|s3://compartiment...|[2.

In [26]:
data.write.mode('overwrite').parquet("s3://compartimentduprojet8/resultas")
print("Spark application successfully finished.")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Spark application successfully finished.